# 🥉 STEP 1 — BRONZE LAYER: RAW INGESTION
Read raw CSV and JSON files into DataFrames and store them as Delta tables in the bronze layer.


In [0]:
# Read raw files
patients_df = spark.read.csv("/FileStore/tables/patients.csv", header=True, inferSchema=True)
hospitals_df = spark.read.json("/FileStore/tables/hospitals.json")
appointments_df = spark.read.csv("/FileStore/tables/appointments_day1.csv", header=True, inferSchema=True)

# Write as Delta tables
patients_df.write.format("delta").mode("overwrite").saveAsTable("bronze_patients")
hospitals_df.write.format("delta").mode("overwrite").saveAsTable("bronze_hospitals")
appointments_df.write.format("delta").mode("overwrite").saveAsTable("bronze_appointments")


# 🥈 STEP 2 — SILVER LAYER: CLEANING & TRANSFORMATION
Clean and transform the data by removing pending appointments, joining tables, and adding year/month columns.


In [0]:
from pyspark.sql.functions import year, month, col

# Load bronze data
patients = spark.table("bronze_patients")
hospitals = spark.table("bronze_hospitals")
appointments = spark.table("bronze_appointments")

# Filter out pending
appointments_filtered = appointments.filter(appointments.status != "Pending")

# Rename duplicate column before join
patients = patients.withColumnRenamed("region", "patient_region")
hospitals = hospitals.withColumnRenamed("region", "hospital_region")

# Join + Add Year/Month
silver_df = (appointments_filtered
    .join(patients, "patient_id", "left")
    .join(hospitals, "hospital_id", "left")
    .withColumn("year", year(col("appointment_date")))
    .withColumn("month", month(col("appointment_date")))
)

# Save
silver_df.write.format("delta").mode("overwrite").saveAsTable("silver_appointments")


# 🥇 STEP 3 — GOLD LAYER: ANALYTICAL AGGREGATIONS
Create analytical summary tables for revenue, patient counts, and top diagnoses.


In [0]:
from pyspark.sql.functions import sum, count, desc

# Load silver table
silver = spark.table("silver_appointments")

# Total revenue per hospital
revenue_per_hospital = silver.groupBy("hospital_name") \
    .agg(sum("cost").alias("total_revenue"))

# Total patients per region (using patient_region)
patients_per_region = silver.groupBy("patient_region") \
    .agg(count("patient_id").alias("total_patients"))

# Top 3 diagnoses by total cost
top3_diagnosis = silver.groupBy("diagnosis") \
    .agg(sum("cost").alias("total_cost")) \
    .orderBy(desc("total_cost")) \
    .limit(3)

# Save all gold tables separately
revenue_per_hospital.write.format("delta").mode("overwrite").saveAsTable("gold_revenue_per_hospital")
patients_per_region.write.format("delta").mode("overwrite").saveAsTable("gold_patients_per_region")
top3_diagnosis.write.format("delta").mode("overwrite").saveAsTable("gold_top3_diagnosis")


# 🔁 STEP 4 — INCREMENTAL LOAD SIMULATION
Load new appointment data (Day 2), filter out pending records, and upsert into the silver table using Delta MERGE.


In [0]:
from delta.tables import DeltaTable

# Read new file
new_appointments = spark.read.csv(
    "/FileStore/tables/appointments_day2.csv",
    header=True,
    inferSchema=True
)

# Filter out Pending records (if any)
new_appointments_filtered = new_appointments.filter(
    new_appointments.status != "Pending"
)

# Load silver Delta table
silver_table = DeltaTable.forName(spark, "silver_appointments")

# Define column mapping
update_dict = {
    "appointment_id": "source.appointment_id",
    "patient_id": "source.patient_id",
    "hospital_id": "source.hospital_id",
    "appointment_date": "source.appointment_date",
    "diagnosis": "source.diagnosis",
    "cost": "source.cost",
    "status": "source.status"
}

# Upsert new data
silver_table.alias("target").merge(
    new_appointments_filtered.alias("source"),
    "target.appointment_id = source.appointment_id"
).whenMatchedUpdate(
    set=update_dict
).whenNotMatchedInsert(
    values=update_dict
).execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

# ✅ STEP 5 — VERIFY INCREMENTAL CHANGES
Check if new data was merged successfully by counting rows and recalculating total revenue.


In [0]:
# Reload updated silver table
silver_updated = spark.table("silver_appointments")

# Check total rows after merge
print("Total rows after incremental load:", silver_updated.count())

# Recalculate revenue to see the difference
updated_revenue = silver_updated.groupBy("hospital_name").agg(sum("cost").alias("total_revenue"))
display(updated_revenue)


Total rows after incremental load: 6


hospital_name,total_revenue
City Care,400
CureWell,300
LifePlus,250
null,1850


# 🕒 STEP 6 — DELTA LAKE FEATURES
Explore key Delta Lake capabilities: Time Travel, Vacuum, and Optimize with Z-Ordering.


In [0]:
# Time Travel: view gold table before incremental load
display(
    spark.sql(
        "SELECT * FROM gold_revenue_per_hospital VERSION AS OF 0"
    )
)

# Vacuum: clean up old versions (recommended minimum is 168 hours)
spark.sql(
    "VACUUM gold_revenue_per_hospital RETAIN 168 HOURS"
)

# Optimize + Z-ORDER
spark.sql(
    "OPTIMIZE gold_revenue_per_hospital ZORDER BY (hospital_name)"
)

hospital_name,total_revenue
City Care,400
CureWell,300
LifePlus,250


DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

# 📊 STEP 7 — ANALYTICAL QUESTIONS
Answer key business questions using SQL queries on the Silver table.


In [0]:
#  Total revenue generated by each hospital
spark.sql("SELECT hospital_name, SUM(cost) AS total_revenue FROM silver_appointments GROUP BY hospital_name").show()

#  Average cost per diagnosis
spark.sql("SELECT diagnosis, AVG(cost) AS avg_cost FROM silver_appointments GROUP BY diagnosis").show()

#  Number of patients served per region
spark.sql("SELECT patient_region, COUNT(DISTINCT patient_id) AS total_patients FROM silver_appointments GROUP BY patient_region").show()

#  Trend of appointments month-over-month
spark.sql("SELECT year, month, COUNT(appointment_id) AS total_appointments FROM silver_appointments GROUP BY year, month ORDER BY year, month").show()

#  Top 5 most expensive treatments (last 6 months)
spark.sql("SELECT diagnosis, SUM(cost) AS total_cost FROM silver_appointments GROUP BY diagnosis ORDER BY total_cost DESC LIMIT 5").show()


+-------------+-------------+
|hospital_name|total_revenue|
+-------------+-------------+
|    City Care|          400|
|     CureWell|          300|
|     LifePlus|          250|
|         NULL|         1850|
+-------------+-------------+

+-------------+--------+
|    diagnosis|avg_cost|
+-------------+--------+
|          Flu|   250.0|
|     Diabetes|   425.0|
|      Allergy|   300.0|
|Heart Disease|  1200.0|
|         Cold|   200.0|
+-------------+--------+

+--------------+--------------+
|patient_region|total_patients|
+--------------+--------------+
|          NULL|             3|
|          West|             1|
|         North|             1|
|         South|             1|
+--------------+--------------+

+----+-----+------------------+
|year|month|total_appointments|
+----+-----+------------------+
|NULL| NULL|                 3|
|2024|    1|                 3|
+----+-----+------------------+

+-------------+----------+
|    diagnosis|total_cost|
+-------------+----------+
|H